In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf

import os
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = pd.read_csv('/content/drive/My Drive/FutureTech.csv', parse_dates=['datetime'])
yval = df['RPM'] == 0
df['yval'] = yval
df = df[df['machineID']!=4]
length = len(df)

In [ ]:
df.head(90000)

,datetime,machineID,Voltage,RPM,pressure,vibration,yval
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,False
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,False
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,False
...,...,...,...,...,...,...,...
43800,2016-01-01 02:00:00,5,162.964887,511.046527,153.187090,41.772148,False
43801,2016-01-01 03:00:00,5,136.402165,402.228972,152.191926,38.078241,False
43802,2016-01-01 04:00:00,5,181.426439,460.280665,142.451849,44.551477,False
43803,2016-01-01 05:00:00,5,178.261658,424.821800,156.228631,39.976910,False


In [ ]:
def cal_hrs(length, yval):
  hoursto = np.zeros(length)
  hourssince = np.zeros(length)
  l = len(yval)
  temp1 = None
  temp2 = None
  for i in range(length-1, -1, -1):
    if yval[i] == 1:
      temp1 = 1
      hoursto[(i)] = 0
      continue
    if temp1 == 1:
      hoursto[(i)] = hoursto[(i)+1] + 1
    else:
      hoursto[(i)] = None
    if yval[length-1-(i)] == 1:
      temp2 = 1
      hourssince[length-1-(i)] = 0
      continue
    if temp2 == 1:
      hourssince[length-1-(i)] = hourssince[length-1-(i)-1] + 1
    else:
      hourssince[length-1-(i)] = None
  return hoursto, hourssince

In [ ]:
since = np.zeros(0)
to = np.zeros(0)

for mid in df.machineID.unique():
  l = len(df[df.machineID == mid])
  yval = df.yval[df.machineID == mid].values

  hrsince = np.zeros(l)
  hrsto = np.zeros(l)

  temp1 = -1
  temp2 = -1
  
  for i in range(0, l):
    if yval[i] == 1:
      temp1 = 1
      hrsince[i] == 0
    elif temp1 == 1:
      hrsince[i] = hrsince[i-1] + 1

    if yval[l-i-1] == 1:
      temp2 = 1
      hrsto[l-i-1] = 0
    elif temp2 == 1:
      hrsto[l-i-1] = hrsto[l-i] + 1

  temp1 = -1
  temp2 = -1

  for i in range(0, l):
    if hrsto[i] == 0 and hrsto[i-1] == 0:
      temp2 = 1
      hrsto[i] = 720
    elif temp2 == 1:
      hrsto[i] = hrsto[i-1] - 1
    
    if hrsince[l-i-1] == 0 and hrsince[l-i] == 0:
      temp1 = 1
      hrsince[l-i-1] = 720
    elif temp1 == 1:
      hrsince[l-i-1] = hrsince[l-i] - 1

  since = np.append(since, hrsince)
  to = np.append(to, hrsto)

df['hourssince'] = since
df['hoursto'] = to

In [ ]:
print(df)

                 datetime  machineID     Voltage  ...   yval  hourssince  hoursto
0     2015-01-01 06:00:00          1  176.217853  ...  False       642.0     79.0
1     2015-01-01 07:00:00          1  162.879223  ...  False       643.0     78.0
2     2015-01-01 08:00:00          1  170.989902  ...  False       644.0     77.0
3     2015-01-01 09:00:00          1  162.462833  ...  False       645.0     76.0
4     2015-01-01 10:00:00          1  157.610021  ...  False       646.0     75.0
...                   ...        ...         ...  ...    ...         ...      ...
43800 2016-01-01 02:00:00          5  162.964887  ...  False         2.0    719.0
43801 2016-01-01 03:00:00          5  136.402165  ...  False         3.0    718.0
43802 2016-01-01 04:00:00          5  181.426439  ...  False         4.0    717.0
43803 2016-01-01 05:00:00          5  178.261658  ...  False         5.0    716.0
43804 2016-01-01 06:00:00          5  178.789197  ...  False         6.0    715.0

[35044 rows x 9

In [ ]:
features_considered = ['datetime','machineID','Voltage','RPM','pressure','vibration','hourssince','hoursto']
features = df[features_considered]
# features.index = df.index
features.head()

,datetime,machineID,Voltage,RPM,pressure,vibration,hourssince,hoursto
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,642.0,79.0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,643.0,78.0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,644.0,77.0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,645.0,76.0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,646.0,75.0


In [ ]:
TRAIN_SPLIT = 30000
dataset = features.values

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
past_history = 744
future_target = -1
STEP = 1
print(len(dataset))

35044


In [ ]:
x_train_single, y_train_single = multivariate_data(dataset[:, 1:7], dataset[:, 7], 0,
                                                 TRAIN_SPLIT-future_target+1, past_history,
                                                 future_target, STEP, True)
x_val_single, y_val_single = multivariate_data(dataset[:, 1:7], dataset[:, 7],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP, True)
for i in y_train_single:
  if i > 4000:
    print('we got fucked')

In [ ]:
print(x_train_single[0,0])
print(y_train_single[0])

[1 176.217853 418.5040782 113.0779355 45.08768576 642.0]
201.0


In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single.astype(np.float32), y_train_single.astype(np.float32)))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single.astype(np.float32), y_val_single.astype(np.float32)))
val_data_single = val_data_single.batch(BATCH_SIZE)

In [ ]:
tf.keras.backend.clear_session()
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.Flatten())
single_step_model.add(tf.keras.layers.Dense(600,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(400,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(200,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(100,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(64,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(16,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(4,activation='relu'))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-8), loss=tf.keras.losses.MeanSquaredError())

In [ ]:
EVALUATION_INTERVAL = 300
EPOCHS = 800

In [ ]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                          validation_data=val_data_single)

Epoch 1/800
458/458 [==============================] - 3s 8ms/step - loss: 452951.7812 - val_loss: 312731.2812
Epoch 2/800
458/458 [==============================] - 3s 7ms/step - loss: 442031.5625 - val_loss: 302786.1875
Epoch 3/800
458/458 [==============================] - 3s 7ms/step - loss: 432226.3438 - val_loss: 293935.8125
Epoch 4/800
458/458 [==============================] - 3s 7ms/step - loss: 423652.8125 - val_loss: 286308.0000
Epoch 5/800
458/458 [==============================] - 3s 7ms/step - loss: 416343.8125 - val_loss: 279945.7812
Epoch 6/800
458/458 [==============================] - 3s 7ms/step - loss: 410285.0625 - val_loss: 274828.2500
Epoch 7/800
458/458 [==============================] - 3s 7ms/step - loss: 405313.6875 - val_loss: 270756.5312
Epoch 8/800
458/458 [==============================] - 3s 7ms/step - loss: 401190.0625 - val_loss: 267453.9062
Epoch 9/800
458/458 [==============================] - 3s 7ms/step - loss: 397734.7188 - val_loss: 264744.4688
E

In [ ]:
plot_train_history(single_step_history, 'single-Step Training and validation loss')

In [ ]:
y1 = None
y0=None
y=None
for x, y in val_data_single.take(100):
  if y1 != None:
    y0 = single_step_model.predict(x)
    print(y0)
    print(y)
    y_1 = y.numpy()
    #plt.rcParams["figure.figsize"] = (10, 10)

# create the plot space upon which to plot the data
    fig, ax = plt.subplots()

# add the x-axis and the y-axis to the plot
    ax.plot(np.arange(len(y_1)), y0, color="red")
    ax.plot(np.arange(len(y_1)), y, color="blue")
    ax.legend(['predicted','actual'])
# set plot title
    ax.set(title="prediction chart")

# add labels to the axes
    ax.set(xlabel="integer", ylabel="Hours");
    #z = [y1.numpy(), y[BUFFER_SIZE-1].numpy(), y0]
    #show_plot(z, 0, 'Prediction')
  y1 = y